In [ ]:
# # FASE 4: Visualización e Inteligencia de Negocio
# ## Juanes (Colaborador) - Creación de Gráficos y Dashboard

# Instalación de dependencias
# %%
!pip install -q wordcloud plotly

# Configurar estilo
# %%
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from wordcloud import WordCloud
from collections import Counter

plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

# Cargar datos con análisis de sentimiento
# %%
df = pd.read_csv("../data/processed/reviews_with_sentiment.csv")
df.head()

# Cargar top palabras
# %%
texto_global = " ".join(df["texto_limpio"])
frecuencia_palabras = Counter(texto_global.split())
top_palabras = frecuencia_palabras.most_common(10)

# 1. WORDCLOUD (Nube de Palabras)

# Combinar todos los textos limpios
# %%
texto_wordcloud = " ".join(df["texto_limpio"])

# Configurar stopwords adicionales para el wordcloud
# %%
stopwords_extra = {
    "muy", "todo", "mas", "menos", "hacer", "ser", "tener"
}

# Crear wordcloud
# %%
wordcloud = WordCloud(
    width=900,
    height=450,
    background_color="white",
    stopwords=stopwords_extra,
    colormap="viridis"
).generate(texto_wordcloud)


# Visualizar wordcloud
# %%
plt.figure(figsize=(12,6))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.title("Nube de Palabras – Opiniones de Usuarios")
plt.show()


# 2. GRÁFICO DE BARRAS - Top 10 palabras

# Preparar datos para top 10 palabras
# %%
palabras, valores = zip(*top_palabras)

# Crear gráfico de barras horizontal
plt.figure(figsize=(8,5))
bars = plt.barh(palabras, valores)
plt.title("Top 10 Palabras Más Frecuentes")
plt.xlabel("Frecuencia")
plt.gca().invert_yaxis()

# Añadir etiquetas de valor
for bar in bars:
    plt.text(bar.get_width() + 1, bar.get_y() + 0.3,
             str(bar.get_width()))
plt.show()


# 3. DISTRIBUCIÓN DE SENTIMIENTOS

# Gráfico de tarta - Distribución de sentimientos
# %%
sent_counts = df["sentimiento"].value_counts()

axes[0].pie(
    sent_counts,
    labels=sent_counts.index,
    autopct="%1.1f%%",
    startangle=90
)
axes[0].set_title("Distribución de Sentimientos")

# Crear figura con subplots
fig, axes = plt.subplots(1, 2, figsize=(12,5))

# Mejorar etiquetas
axes[0].set_ylabel("")
axes[0].set_xlabel("")
axes[0].set_aspect("equal")

axes[1].set_xlabel("Sentimiento")
axes[1].set_ylabel("Número de comentarios")

axes[0].set_title("Distribución porcentual de sentimientos", fontsize=12)
axes[1].set_title("Frecuencia absoluta de sentimientos", fontsize=12)

plt.suptitle(
    "Análisis Global de Sentimientos de los Usuarios",
    fontsize=14,
    fontweight="bold"
)

# Gráfico de barras
sns.barplot(
    x=sent_counts.index,
    y=sent_counts.values,
    ax=axes[1]
)
axes[1].set_title("Número de Comentarios por Sentimiento")

# Añadir valores en las barras
# %%
for i, v in enumerate(sent_counts.values):
    axes[1].text(i, v + 1, str(v), ha="center")

plt.tight_layout()
plt.show()

# 4. ANÁLISIS DE PUNTUACIÓN VS SENTIMIENTO
   
    # Añadir línea de regresión
    
plt.figure(figsize=(8,5))
sns.regplot(
    x="puntuacion",
    y="polaridad",
    data=df,
    scatter_kws={"alpha":0.6}
)

    # Añadir anotación de correlación
corr = df["puntuacion"].corr(df["polaridad"])
plt.title(f"Relación Puntuación vs Polaridad (corr={corr:.2f})")
plt.show()

   
# 5. GRÁFICOS ADICIONALES DE VALOR

# 5.1 Distribución de longitud de reseñas por sentimiento

# Boxplot de longitud por sentimiento
# %%
plt.figure(figsize=(8,5))
sns.boxplot(
    x="sentimiento",
    y="num_palabras",
    data=df
)
plt.title("Longitud de Reseñas por Sentimiento")
plt.show()

# Histograma de polaridad
sns.histplot(df["polaridad"], bins=30, kde=True)
plt.title("Distribución de Polaridad")
plt.show()

# 5.2 Heatmap de palabras más frecuentes por sentimiento

# Crear DataFrame para heatmap
palabras_heatmap = {}

for sentimiento in df["sentimiento"].unique():
    texto = " ".join(df[df["sentimiento"] == sentimiento]["texto_limpio"])
    palabras_heatmap[sentimiento] = dict(Counter(texto.split()).most_common(5))

# Crear heatmap
plt.figure(figsize=(8,5))
sns.heatmap(df_heatmap, annot=True, cmap="YlGnBu")
plt.title("Palabras Más Frecuentes por Sentimiento")
plt.show()

# 5.3 Gráfico de evolución temporal (si hay fechas)
if "fecha" in df.columns:
    df["fecha"] = pd.to_datetime(df["fecha"])
    df_time = df.groupby("fecha")["polaridad"].mean()

    fig, ax1 = plt.subplots(figsize=(10,5))
    ax1.plot(df_time.index, df_time.values)
    ax1.set_ylabel("Polaridad media")
    ax1.set_title("Evolución Temporal del Sentimiento")
    plt.show()

# 6. DASHBOARD INTERACTIVO (OPCIONAL CON PLOTLY)
dashboard = make_subplots(
    rows=2,
    cols=2,
    subplot_titles=(
        "Distribución de Sentimientos",
        "Top 10 Palabras",
        "Polaridad vs Puntuación",
        "Longitud de Reseñas por Sentimiento"
    )
)

    # 1. Gráfico de tarta (sentimientos)
sent_counts = df["sentimiento"].value_counts()

dashboard.add_trace(
    go.Pie(
        labels=sent_counts.index,
        values=sent_counts.values,
        hole=0.3
    ),
    row=1,
    col=1
)

    # 2. Gráfico de barras (top palabras)
palabras, valores = zip(*top_palabras)

dashboard.add_trace(
    go.Bar(
        x=valores,
        y=palabras,
        orientation="h"
    ),
    row=1,
    col=2
)

    # 3. Gráfico de dispersión (polaridad vs puntuación)
    # %%
dashboard.add_trace(
    go.Scatter(
        x=df["puntuacion"],
        y=df["polaridad"],
        mode="markers",
        marker=dict(
            size=8,
            opacity=0.6
        ),
        text=df["sentimiento"]
    ),
    row=2,
    col=1
)
        
    # 4. Boxplot (longitud por sentimiento)
    # %%
for sentimiento in df["sentimiento"].unique():
    dashboard.add_trace(
        go.Box(
            y=df[df["sentimiento"] == sentimiento]["num_palabras"],
            name=sentimiento
        ),
        row=2,
        col=2
    )
        
    # Actualizar layout
    # %%
dashboard.update_layout(
    title="Dashboard Interactivo – Inteligencia de Opinión",
    height=800,
    showlegend=True,
    template="plotly_white"
)

    # Guardar dashboard interactivo
    # %%
dashboard.write_html("../data/dashboard_interactivo.html")
dashboard.show()

# 7. INFORME VISUAL FINAL

# Crear figura resumen
# %%
fig = plt.figure(figsize=(14,10))
gs = fig.add_gridspec(2, 3)

# Definir grid para subplots
fig = plt.figure(figsize=(16, 10))

grid = fig.add_gridspec(
    nrows=2,
    ncols=3,
    hspace=0.35,
    wspace=0.25
)

# 1. WordCloud
ax1 = fig.add_subplot(gs[0, 0])
ax1.imshow(wordcloud)
ax1.axis("off")
ax1.set_title("WordCloud")

# 2. Distribución de sentimientos
ax2 = fig.add_subplot(gs[0, 1])
sent_counts.plot(kind="bar", ax=ax2)
ax2.set_title("Sentimientos")

# 3. Top 10 palabras
ax3 = fig.add_subplot(gs[0, 2])
ax3.barh(palabras, valores)
ax3.set_title("Top Palabras")

# 4. Polaridad vs Puntuación
ax4 = fig.add_subplot(gs[1, 0])
sns.regplot(x="puntuacion", y="polaridad", data=df, ax=ax4)

# 5. Distribución de polaridad
ax5 = fig.add_subplot(gs[1, 1])
sns.histplot(df["polaridad"], ax=ax5)

# 6. Longitud por sentimiento
ax6 = fig.add_subplot(gs[1, 2])
sns.boxplot(x="sentimiento", y="num_palabras", data=df, ax=ax6)

# Añadir texto con insights
fig.suptitle(
    "Informe Visual Final – Inteligencia de Opinión",
    fontsize=16
)
plt.tight_layout()
plt.show()

# RESUMEN FINAL
# %%
print("""
FASE 4 COMPLETADA:
- Visualización clara y orientada a negocio
- Gráficos descriptivos y analíticos
- Relación sentimiento–puntuación demostrada
- Dashboard interactivo generado
""")


In [ ]:

# %%
# VISUALIZACIÓN DE INTELIGENCIA DE USUARIOS (Persona B)
import seaborn as sns
import matplotlib.pyplot as plt

def plot_user_intelligence(df):
    plt.figure(figsize=(15, 10))
    
    # 1. Top Ubicaciones
    plt.subplot(2, 2, 1)
    if 'ubicacion' in df.columns:
        df['ubicacion'].value_counts().head(10).plot(kind='bar', color='skyblue')
        plt.title('Top 10 Ubicaciones de los Usuarios')
        plt.ylabel('Cantidad de Reseñas')
    
    # 2. Segmentos de Usuario
    plt.subplot(2, 2, 2)
    if 'segmento_usuario' in df.columns:
        df['segmento_usuario'].value_counts().plot(kind='pie', autopct='%1.1f%%', colors=sns.color_palette('pastel'))
        plt.title('Distribución de Segmentos de Usuario')
        plt.ylabel('')
    
    # 3. Polaridad por Segmento
    plt.subplot(2, 2, 3)
    if 'segmento_usuario' in df.columns:
        sns.barplot(x='segmento_usuario', y='polarity', data=df, palette='viridis', errorbar=None)
        plt.title('Polaridad Media por Segmento de Usuario')
        plt.xticks(rotation=45)
    
    # 4. Relación Experiencia vs Puntuación
    plt.subplot(2, 2, 4)
    if 'num_resenas_usuario_total' in df.columns:
        plt.scatter(df['num_resenas_usuario_total'], df['puntuacion'], alpha=0.5)
        plt.title('Experiencia (Total Reseñas) vs Puntuación')
        plt.xlabel('Reseñas totales del usuario')
        plt.ylabel('Estrellas')
        plt.xscale('log')
    
    plt.tight_layout()
    plt.savefig('../visualizations/user_intelligence_dashboard.png')
    plt.show()

# Ejecutar visualización si existen los datos
if 'segmento_usuario' in df.columns or 'ubicacion' in df.columns:
    plot_user_intelligence(df)
else:
    print("Datos de usuario no encontrados para visualizar. Ejecuta el pipeline completo.")
